# Information Retrieval Notebook 2

In [1]:
# no time to lose:
#!wget https://gerdes.fr/saclay/informationRetrieval/our_msmarco.zip
#!unzip our_msmarco.zip
# this will be big: 1.2gb!
# you will get three files 

In [2]:
# this turns on the autotimer, so that every cell has a timing information below
try:
    %load_ext autotime
except:
    #!pip install ipython-autotime
    %load_ext autotime
# stop using:
# %unload_ext autotime
#!pip install sklearn

time: 0 ns (started: 2024-03-24 19:30:29 +01:00)


In [3]:
#!pip install scikit-learn
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import nltk
import re
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from collections import Counter
from tqdm.notebook import tqdm
#from sklearn.feature_extraction.text import CountVectorizer
import dask.dataframe as dd

time: 8.98 s (started: 2024-03-24 19:30:30 +01:00)


## Our dataset

- "TREC stands for the Text Retrieval Conference. Started in 1992 it is a series of workshops that focus on supporting research within the information retrieval community. It provides the infrastructure necessary for large-scale evaluation of text retrieval methodologies. Every year these workshops are organized, which are centered around a set of tracks. These tracks encourage new researches in the area of information retrieval."
- TREC 2019 Deep Learning Track https://microsoft.github.io/msmarco/TREC-Deep-Learning-2019
- data from MS-Marco https://microsoft.github.io/msmarco/
- The dataset contains  367k queries and a corpus of 3.2 million documents. 
___
- if you want to reproduce my selection or get a bigger set, uncomment and execute


In [128]:
#!wget https://msmarco.blob.core.windows.net/msmarcoranking/msmarco-docs.tsv.gz
#!wget https://msmarco.blob.core.windows.net/msmarcoranking/msmarco-doctrain-queries.tsv.gz
#!wget https://msmarco.blob.core.windows.net/msmarcoranking/msmarco-doctrain-top100.gz
	
#!gzip -d msmarco-docs.tsv.gz
#!gzip -d msmarco-doctrain-queries.tsv.gz
#!gzip -d msmarco-doctrain-top100.gz


time: 0 ns (started: 2024-03-24 09:12:49 +01:00)


## Skip to the section "reading in our smaller files"

come back here if you want to understand how to reduce the dataset or you want to build your own dataset

- we have three datasets:
    
    1. the queries: msmarco-doctrain-queries.tsv
    2. the gold: msmarco-doctrain-top100.tsv is a table containing query_id, doc_id and rank
    3. the actual documents: msmarco-docs.tsv 21GB of text! doc_id, url, title, text

In [129]:
#all_queries=pd.read_table('msmarco-doctrain-queries.tsv',header=None)
#all_queries.columns=['qid','query']
#print('Shape=>',all_queries.shape)
#all_queries.head()

time: 0 ns (started: 2024-03-24 09:12:49 +01:00)


#### reducing the dataset
- here we take 1000 queries. 
- if this is too big for your computer, use this code to build smaller version, starting with the already reduced 1000 query set that we've downloaded before


In [130]:
#our_queries=all_queries.sample(n=1000,random_state=42).reset_index(drop=True)
#print('Shape=>',our_queries.shape)
#our_queries.head()

time: 0 ns (started: 2024-03-24 09:12:49 +01:00)


In [131]:
#our_queries.to_csv('our.msmarco.queries.tsv',sep='\t')

time: 0 ns (started: 2024-03-24 09:12:49 +01:00)


#### the gold file
- 36m lines!

In [132]:
"""
gold_top100=pd.read_table('msmarco-doctrain-top100',delimiter=' ',header=None)
gold_top100.columns=['qid','Q0','docid','rank','score','runstring']
print('Shape=>',gold_top100.shape)
display(gold_top100.head())
# Reducing train_top100 for training
our_gold_top100=train_top100[gold_top100['qid'].isin(our_queries['qid'].unique())].reset_index(drop=True)
print('Shape=>',our_gold_top100.shape)
our_gold_top100.head()
"""

"\ngold_top100=pd.read_table('msmarco-doctrain-top100',delimiter=' ',header=None)\ngold_top100.columns=['qid','Q0','docid','rank','score','runstring']\nprint('Shape=>',gold_top100.shape)\ndisplay(gold_top100.head())\n# Reducing train_top100 for training\nour_gold_top100=train_top100[gold_top100['qid'].isin(our_queries['qid'].unique())].reset_index(drop=True)\nprint('Shape=>',our_gold_top100.shape)\nour_gold_top100.head()\n"

time: 0 ns (started: 2024-03-24 09:12:49 +01:00)


In [133]:
#our_gold_top100.to_csv('our.msmarco.gold.tsv',sep='\t')

time: 0 ns (started: 2024-03-24 09:12:49 +01:00)


#### the data file

- it's so big that it's smarter to use dask: https://docs.dask.org/en/stable/

In [134]:
#df=dd.read_table('msmarco-docs.tsv',blocksize=100e6,header=None) #  partitions of 100MB
#df.columns=['docid','url','title','body']
#df.head()

time: 0 ns (started: 2024-03-24 09:12:49 +01:00)


In [135]:
# can't get the number of rows quickly :s
# very slow:
# len(df.index)

# faster:
#!wc -l msmarco-docs.tsv

time: 0 ns (started: 2024-03-24 09:12:50 +01:00)


- big dataset with 3m rows!
- we want the top 100 for our queries
- this takes some time!

In [136]:
def create_corpus(result):
  unique_docid=result['docid'].unique()
  condition=df['docid'].isin(unique_docid)
  corpus=df[condition].reset_index(drop=True)
  corpus=corpus.drop(columns='url')
  print('Number of Rows=>',len(corpus))
  return corpus

#our_docs=create_corpus(our_gold_top100)
#our_docs.head()

time: 0 ns (started: 2024-03-24 09:12:50 +01:00)


In [137]:
#our_docs.to_csv('our.msmarco.docs.tsv',sep='\t', single_file=True)

time: 0 ns (started: 2024-03-24 09:12:50 +01:00)


- this is still a big file: 92k documents

## Reading in our smaller files

Here we use the following commands:

- !wget https://gerdes.fr/saclay/informationRetrieval/our_msmarco.zip
- !unzip our_msmarco.zip

- What is the MS MARCO Dataset?

The MS MARCO (Microsoft Machine Reading Comprehension) dataset is a large-scale dataset for machine reading comprehension and information retrieval. It's built from real-world data and includes a variety of questions that people have asked using Bing search, paired with passages that a human reading comprehension model has determined might contain answers to these questions.

The dataset includes three main types of data:

1. **Questions**: Real anonymized questions from Bing users. Each question is naturally phrased. # The questions are in the our.msmarco.queries.tsv file

2. **Passages**: Excerpts from web pages that could potentially answer the questions. # The questions are in the our.msmarco.queries.tsv file

3. **Relevance Labels**: Information about whether a given passage is relevant to the corresponding question. # The questions are in the our.msmarco.gold.tsv file

An example entry in the MS MARCO dataset might look like this:

- **Question**: "What is a neural network?"
- **Passage**: "A neural network is a network or circuit of neurons, or in a modern sense, an artificial neural network, composed of artificial neurons or nodes. Thus a neural network is either a biological neural network, made up of real biological neurons, or an artificial neural network, for solving artificial intelligence (AI) problems."
- **Relevance Label**: Rank + Score


In [4]:
queries = pd.read_csv('our_msmarco/our.msmarco.queries.tsv',sep='\t',usecols=[1,2])
queries

,qid,query
0,687888,what is a jpe
1,480210,price for asphalt driveway
2,591004,what causes pressure skin bruising
3,260536,how long drive from flagstaff to grand canyon
4,39422,average number of bowel movements per day for ...
...,...,...
995,89597,cell voltage mv meaning
996,1167043,what an ip address
997,737304,what is daily max citizens atm
998,156934,do i need a florida commercial driver license


time: 47 ms (started: 2024-03-24 19:30:40 +01:00)


In [5]:
gold = pd.read_csv('our_msmarco/our.msmarco.gold.tsv',sep='\t',usecols=[1,3,4,5])
gold

,qid,docid,rank,score
0,310290,D579750,1,-5.11498
1,310290,D579754,2,-5.57703
2,310290,D2380815,3,-5.84852
3,310290,D822566,4,-5.95002
4,310290,D2249695,5,-6.08326
...,...,...,...,...
99995,257942,D253854,96,-6.32693
99996,257942,D3056621,97,-6.32837
99997,257942,D1323491,98,-6.32871
99998,257942,D2722485,99,-6.33100


time: 266 ms (started: 2024-03-24 19:30:40 +01:00)


In [6]:
docs = pd.read_csv('our_msmarco/our.msmarco.docs.tsv',sep='\t',usecols=[1,2,3])
docs

,docid,title,body
0,D2981241,What do you call a group of lions?,Lions Vocabulary of the English Language Word ...
1,D687756,.,"The A Priori Argument ( also, Rationalization;..."
2,D913099,Everything You Need To Learn How To Cook Veget...,Home > How To Cook Vegetables Everything You N...
3,D328017,"What is the difference between latitude, longi...",Longitude Latitude Geographic Coordinate Syste...
4,D1636347,When was the pulley invented?,Answers.com ® Wiki Answers ® Categories Techno...
...,...,...,...
92560,D3379210,Top 39 Doctor insights on: Can An Iud Cause Ha...,Top 39 Doctor insights on: Can An Iud Cause Ha...
92561,D3068739,How to get back your DirecTV cancellation fees,How to get back your Direc TV cancellation fee...
92562,D1590402,Certification FAQs,Fingerprinting 1. Where can I get fingerprinte...
92563,D2175490,Greenhouse gas emissions by Canadian economic ...,"Access PDF (682 KB)In 2015, Canada's total gre..."


time: 43.4 s (started: 2024-03-24 19:30:41 +01:00)


In [141]:
docs.iloc[0]["body"]

'Lions Vocabulary of the English Language Word Definitions, Terminology, and Jargon Animals English (language)What do you call a group of lions?17 Answers Michael Scofields, Expert in Language Answered Mar 26, 2016A group of lions is called troop or sault. A family of lions is called pride. Mammals Apes A shrewdness Asses A pace Badgers A cete Bats A colony Bears A sloth, sleuth Buffalo A gang, an obstinacy (I suspect these refer to old world buffalo; use “herd” for American bison)Cats A clowder, a pounce; for kittens…A kindle, litter, an intrigue Cattle A drove, herd Deer A herd, bevy (refers only to roe deer)Dogs A litter (young), pack (wild), cowardice (of curs); specific to hounds…A cry, mute, pack, kennel Elephants A herd Elk A gang Ferrets A business Fox A leash, skulk, earth Giraffes A tower Goats A tribe, trip Gorillas A band Hippopotamuses A bloat Horses A team, harras, rag (for colts), stud (a group of horses belonging to a single owner, string (ponies)Hyenas A cackle Kangaro

time: 15 ms (started: 2024-03-24 09:13:29 +01:00)


In [7]:
# Creating Training Set of Queries
training_queries=queries.iloc[:500]
print('Shape of Training Dataset =>',training_queries.shape)
display(training_queries.head())
# Creating Testing Set of Queries
testing_queries=queries.iloc[500:]
print('Shape of Testing Dataset =>',testing_queries.shape)
testing_queries.head()

Shape of Training Dataset => (500, 2)


,qid,query
0,687888,what is a jpe
1,480210,price for asphalt driveway
2,591004,what causes pressure skin bruising
3,260536,how long drive from flagstaff to grand canyon
4,39422,average number of bowel movements per day for ...


Shape of Testing Dataset => (500, 2)


,qid,query
500,116364,decree verb definition
501,638813,what does hemiballistic mean
502,401631,is advil considered aspirin?
503,1050265,who sang the song midnight train to georgia
504,632336,what does aq mean in chemistry


time: 31 ms (started: 2024-03-24 19:31:24 +01:00)


## Exploring the data

### 🚧 Todo: Check whether there are NaN and take care of them

#### Checking in Queries

In [8]:
# Check for NaN values
print("Found NaN Values:\n\n", queries.isna().any(), "\n")

# Get the count of NaN values per column
print("Total Count NaN Values:\n\n", queries.isna().sum())

Found NaN Values:

 qid      False
query    False
dtype: bool 

Total Count NaN Values:

 qid      0
query    0
dtype: int64
time: 32 ms (started: 2024-03-24 19:31:25 +01:00)


#### Checking in Gold

In [9]:
# Check for NaN values
print("Found NaN Values:\n\n", gold.isna().any(), "\n")

# Get the count of NaN values per column
print("Total Count NaN Values:\n\n", gold.isna().sum())

Found NaN Values:

 qid      False
docid    False
rank     False
score    False
dtype: bool 

Total Count NaN Values:

 qid      0
docid    0
rank     0
score    0
dtype: int64
time: 46 ms (started: 2024-03-24 19:31:25 +01:00)


#### Checking in Docs

In [10]:
# Shape Docs
print('Shape of Docs Dataset =>', docs.shape, "\n")

# Check for NaN values
print("Found NaN Values:\n\n", docs.isna().any(), "\n")

# Get the count of NaN values per column
print("Total Count NaN Values:\n\n", docs.isna().sum())

Shape of Docs Dataset => (92565, 3) 

Found NaN Values:

 docid    False
title     True
body      True
dtype: bool 

Total Count NaN Values:

 docid      0
title    372
body       3
dtype: int64
time: 94 ms (started: 2024-03-24 19:31:26 +01:00)


#### Filling with "." titles with NaN values + Removing rows with NaN body

In [11]:
# Filling with "." NaN title
docs['title'] = docs['title'].fillna('.')

# Removing rows with NaN body
docs = docs.dropna(subset=['body'])

# Check for NaN values
print("Found NaN Values:\n\n", docs.isna().any(), "\n")

# Get the count of NaN values per column
print("Total Count NaN Values:\n\n", docs.isna().sum())


Found NaN Values:

 docid    False
title    False
body     False
dtype: bool 

Total Count NaN Values:

 docid    0
title    0
body     0
dtype: int64
time: 188 ms (started: 2024-03-24 19:31:26 +01:00)


### Let's have a look at some random query:

In [147]:
queries.loc[111]

qid                                      251898
query    how long does getting a doctorate take
Name: 111, dtype: object

time: 0 ns (started: 2024-03-24 09:13:30 +01:00)


In [148]:
gold[gold.qid==251898]

,qid,docid,rank,score
36200,251898,D2865964,1,-4.74293
36201,251898,D3557816,2,-4.90695
36202,251898,D2723985,3,-4.95911
36203,251898,D1951655,4,-4.97272
36204,251898,D1709749,5,-5.02176
...,...,...,...,...
36295,251898,D2531901,96,-5.56896
36296,251898,D2956542,97,-5.57138
36297,251898,D301873,98,-5.57262
36298,251898,D2952336,99,-5.57504


time: 31 ms (started: 2024-03-24 09:13:30 +01:00)


### 🚧 Todo: Let's look at the top-ranked document for that query
- title
- body

#### Top-Ranked document: docid = D2865964

In [149]:
from IPython.display import display

top = docs[docs.docid == "D2865964"]

with pd.option_context('display.max_colwidth', None):
    display(docs[docs.docid == "D2865964"])

docid                                                 title  \
56982  D2865964  How long does it take to get a post-doctoral degree?   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

time: 47 ms (started: 2024-03-24 09:13:30 +01:00)


### 🚧 Todo: Let's look at the second document
- Let's make a function to make that easier

In [150]:
def titleAndBody(qid,rank):
    # Merge the 'gold' and 'docs' DataFrames on 'docid'
    merged_df = pd.merge(gold, docs, on='docid', how='inner')
    
    # Filter the merged DataFrame for the given 'qid' and 'rank'
    target_doc = merged_df[(merged_df['qid'] == qid) & (merged_df['rank'] == rank)]
    
    # Check if the target document is found
    if not target_doc.empty:
        # Display the title and body of the document
        print(f"Title: {target_doc.iloc[0]['title']}")
        print(f"Body: {target_doc.iloc[0]['body']}")
    else:
        print("Document not found for the given qid and rank.")


titleAndBody(251898,1)

Title: How long does it take to get a post-doctoral degree?
Body: Answers.com ® Wiki Answers ® Categories Jobs & Education Education College Degrees Graduate Degrees How long does it take to get a post-doctoral degree? Flag How long does it take to get a post-doctoral degree? Answer by Joe Ragusa Confidence votes 98.6KI am not aware of any such thing as a postdoctoral degree. You may be referring to postdoctoral research which may be required for obtaining a tenure-track faculty position, especially at research oriented institutions. Some have suggested that postdoctoral appointments - that were traditionally optional - have become mandatory as demand for tenure-track positions in academia has drastically increased over previous decades (Wikipedia). The length of time in this case depends on the type, content, scope, and depth of the research.2 people found this useful Was this answer useful? Yes Somewhat No Tyler Durden9988 1,326 Contributions How long does it take to get a doctoral d

#### Let's look at the 100th document

In [151]:
titleAndBody(251898,99)

Title: .
Body: Solutions Sang Level 1 (Contributor)61 Answers, 2 Followers"What process does long distance mail go through..."2 0 What process does long distance mail go through? I.e from your mail slot to.... Was this answer helpful? Yes | No Comment Reply Report This answer closely relates to: Calculate how long it will take for mail to be sent from one state to another Mechanicsburg pa to cazenovia ny how long to ship regular mail Whats the longest it take to recieve mail from another state Ellsworth Level 1 (Contributor)52 Answers, 1 Follower0 0 From cerritos ca to cleveland oh. Was this answer helpful? Yes | No Comment Reply Report This answer closely relates to: Mailed from kansas city mo on a friday when can i expect it in cleveland ohio Freida Level 1 (Contributor)13 Answers, 1 Follower1 1 How long does it take to get your state and fedurel tax back? Was this answer helpful? Yes | No Comment Reply Report This answer closely relates to: Approximately how long will it take for a 

### 🚧 todo: try this with a few different queries to get a feel of the quality of the gold

In [152]:
titleAndBody(251898,19)

Title: How long does it take to get a bachelor's in computer engineering?
Body: Answers.com ® Wiki Answers ® Categories Business & Finance Business and Industry Industries and Professions How long does it take to get a bachelor's in computer engineering? Flag How long does it take to get a bachelor's in computer engineering? Edit Answer by Joe Ragusa Confidence votes 98.5KA bachelor's degree in most areas can require between 124 to 128 college credits particular to a specific program of study. This would be based on a college or university that operates on a regular two semester academic year which is the most common. For institutions that operate on a tri-semester or quarter-semester it would be quite different. That being said, the Bachelor's degree - in most cases - is designed as a four year program of study, provided the student is full-time and follows their chosen program of study as prescribed by the college or university, and also provided the individual does not require prere

In [153]:
titleAndBody(251898,53)

Title: .
Body: Q How long does regular mail take with in the same state? How long does it take a regular mail to arrive within the same state in the us Topic: Asked by: Roxie In Politics & Government > Other - Politics & Government > Mail>A Top Solutions1 day if it is within certain tri counties (60 mile radius). 2 days if it`s within the sta ... read more Do not use all of these Private Money Lender here, there are not legitimate and if they ha ... read more Add your answer Post to Facebook Post to Twitter Subscribe me Solutions Avis Level 1 (Contributor)19 Answers, 1 Follower"1 day if it is within certain tri counties (60 mile radius..."5 0 1 day if it is within certain tri counties (60 mile radius ). 2 days if it`s within the state (north-south-east-west). 3 days if it is within a remote area of the same state. If you want it to be priority it will still take 2-3 days. Otherwise across the country it could take 3-5 days. Not including weekends. And for packages 7-10 days. Internatio

In [154]:
docs[docs.docid == "D3381269"]

,docid,title,body
89525,D3381269,.,Q How long does regular mail take with in the ...


time: 31 ms (started: 2024-03-24 09:13:32 +01:00)


## Doing our first baseline retrieval function

- Todo: 
    - Build and fit a binary CountVectorizer on the **titles** - DONE
    - Play with and understand build_analyzer, build_tokenizer, and transform - DONE
    - Transform our query 111 - DONE
    - Understand what happens with yet unseen words in the transform process - DONE
    - Find the docs with the most words in common - DONE
    - Write an evaluation function computing the top 10 precision p@10 - DONE
    - Apply to our 500 queries - DONE


#### Reminder of our documents dataset

In [10]:
docs

,docid,title,body
0,D2981241,What do you call a group of lions?,Lions Vocabulary of the English Language Word ...
1,D687756,.,"The A Priori Argument ( also, Rationalization;..."
2,D913099,Everything You Need To Learn How To Cook Veget...,Home > How To Cook Vegetables Everything You N...
3,D328017,"What is the difference between latitude, longi...",Longitude Latitude Geographic Coordinate Syste...
4,D1636347,When was the pulley invented?,Answers.com ® Wiki Answers ® Categories Techno...
...,...,...,...
92560,D3379210,Top 39 Doctor insights on: Can An Iud Cause Ha...,Top 39 Doctor insights on: Can An Iud Cause Ha...
92561,D3068739,How to get back your DirecTV cancellation fees,How to get back your Direc TV cancellation fee...
92562,D1590402,Certification FAQs,Fingerprinting 1. Where can I get fingerprinte...
92563,D2175490,Greenhouse gas emissions by Canadian economic ...,"Access PDF (682 KB)In 2015, Canada's total gre..."


time: 16 ms (started: 2024-03-24 16:29:07 +01:00)


#### Build and fit a Binary CountVectorizer on the titles

In [156]:
from sklearn.feature_extraction.text import CountVectorizer

titles = docs.title.values
print("List of all titles within docs:\n\n", titles)

List of all titles within docs:

 ['What do you call a group of lions?' '.'
 'Everything You Need To Learn How To Cook Vegetables' ...
 'Certification FAQs'
 'Greenhouse gas emissions by Canadian economic sector'
 'QuickBooks Bill Pay']
time: 63 ms (started: 2024-03-24 09:13:32 +01:00)


In [157]:
# understand the options: 
# https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

# Instantiation of CountVectorizer with binary mode
vectorizer = CountVectorizer(binary=True)

X_titles = vectorizer.fit_transform(titles)

print('We got',len(vectorizer.get_feature_names_out()),'features, for example',vectorizer.get_feature_names_out()[33333:33339])


We got 39479 features, for example ['stable' 'stack' 'stackable' 'stacked' 'stackplayer' 'stacks']
time: 4.08 s (started: 2024-03-24 09:13:32 +01:00)


##### Visualizing the sparse matrix X

In [158]:
subset = X_titles[:100, :100].toarray()
subset

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

time: 31 ms (started: 2024-03-24 09:13:36 +01:00)


#### Play with and understand build_analyzer, build_tokenizer, and transform

Summary:

- build_analyzer: This function returns a callable that prepares the text data by preprocessing, tokenizing, and filtering stop words. It's the entire process that CountVectorizer goes through to convert text into tokens before counting them.

- build_tokenizer: This function returns a callable that splits the input text into tokens. It does not perform any preprocessing or stop word removal.

- transform: This method is used on the CountVectorizer to convert the array of text data (titles) into a matrix of token counts.

In [159]:
print("Original Title:\n", docs.iloc[0].title)

# Get the analyzer function from the CountVectorizer instance
analyzer = vectorizer.build_analyzer()

# Use the analyzer on a title
example_title = docs['title'].iloc[0]
print("\nAnalyzed Title:\n", analyzer(example_title), "\n")  

Original Title:
 What do you call a group of lions?

Analyzed Title:
 ['what', 'do', 'you', 'call', 'group', 'of', 'lions'] 

time: 16 ms (started: 2024-03-24 09:13:36 +01:00)


In [160]:
# Get the tokenizer function from the CountVectorizer instance
tokenizer = vectorizer.build_tokenizer()

# Use the tokenizer on a title
# This will output the raw tokens without preprocessing, however it removes punctuation.
print("Tokenized Title:\n", tokenizer(example_title), "\n")  


Tokenized Title:
 ['What', 'do', 'you', 'call', 'group', 'of', 'lions'] 

time: 16 ms (started: 2024-03-24 09:13:36 +01:00)


In [161]:
# Transform the titles into a sparse matrix of token counts
qv = vectorizer.transform(docs['title'])
qv

<92562x39479 sparse matrix of type '<class 'numpy.int64'>'
	with 500636 stored elements in Compressed Sparse Row format>

time: 1.34 s (started: 2024-03-24 09:13:36 +01:00)


#### Query to transform: 111

##### Information:

Main differences between fit_transform and transform.

1. fit_transform:
    - When applied to training data, fit_transform first 'fits' or 'trains' the transformer on the data. This means it learns any necessary parameters from the data, such as the mean and standard deviation for scaling or the document-term matrix in text vectorization.
    - After fitting, it immediately 'transforms' the training data according to these learned parameters. This means it applies the learned transformation rules to the data, such as scaling the data or converting text data into a numeric format.
    - The result of fit_transform is the transformed version of the input data, ready for training a machine learning model.
    - fit_transform is typically used on the training dataset to prepare the transformer for later use on other data.

2. transform:

    - transform is used after the transformer has been fitted. If you have new data that you want to process in the same way as the training data, you'd use transform.
    - It applies the transformation rules learned during the fit phase to new data. The new data gets transformed without re-learning the transformation parameters.
    - This method ensures that the transformation on new data is consistent with the transformation performed on the training data.
    - transform is used on new or testing datasets to apply the same transformation as learned from the training dataset.

In [162]:
query_111 = [queries.iloc[111].query]

print(query_111)

['how long does getting a doctorate take']
time: 0 ns (started: 2024-03-24 09:13:38 +01:00)


Previously, we fit CountVectorizer with the complete list of titles, now we transform the query 111 by applying the transformation rules learned during the fit phase.

##### Transformed Query

In [163]:
query_111_transformed = vectorizer.transform(query_111)

print(query_111_transformed)

  (0, 11411)	1
  (0, 11430)	1
  (0, 15497)	1
  (0, 17586)	1
  (0, 21277)	1
  (0, 34584)	1
time: 16 ms (started: 2024-03-24 09:13:38 +01:00)


### 🚧 Todo:

- Understand what happens with yet unseen words in the transform process - DONE
- Think of the shape of X, what are the rows, what are the columns? - DONE
- How to select the titles that have the words of our query?
       - think of matrix multiplication and transposition - DONE

#### 1. Understand what happens with yet unseen words in the transform process

Words that were not seen during the fitting process of CountVectorizer will be ignored during the transformation because they are not part of the vocabulary that the vectorizer knows. In other words, if a word was not encountered in the training dataset (during .fit or .fit_transform), it will not be recognized when new data is passed through .transform.

#### 2. Think of the shape of X, what are the rows, what are the columns?

In the output of CountVectorizer, rows represent individual documents (titles), and columns represent unique tokens or words across the entire corpus that was fitted to the vectorizer. Each cell in the matrix contains the count (or presence/absence in binary mode) of the token corresponding to that column in the document corresponding to that row.

#### 3. How to select the titles that have the words of our query?

To select titles containing specific words from a query, we can use matrix multiplication. Multiplying the term-document matrix (where each row represents a document and each column represents a term) by the transpose of the term-query matrix will give you a new matrix indicating which documents contain the terms of the query. Then we select the document which contains the highest number of terms from our query.

Let's consider an example to illustrate the idea:

- document_titles = [
    "How to bake a cake",
    "How to program in Python",
    "The history of ancient Rome"
]

- query = ["how to bake"]

We will use CountVectorizer to create the term-document matrix for the titles and transform the query into its vector representation. Then, multiply both to obtain the common words.

In [164]:
document_titles = [
    "How to bake a cake",
    "How to program in Python",
    "The history of ancient Rome"
]

query = ["how to bake"]

# Initialize the CountVectorizer
vectorizer_2 = CountVectorizer()

# Fit and transform the document titles
X_example = vectorizer_2.fit_transform(document_titles)

# Convert to dense matrix for the example display
dense_X_example = X_example.toarray()

# Show the term-document matrix
print("document_titles vector:\n", dense_X_example)

# Transform the query using the same vectorizer
query_vector = vectorizer_2.transform(query)

# Convert to dense matrix for the example display
dense_query_vector = query_vector.toarray()

# Show the query vector
print("\nquery vector:\n", dense_query_vector)

# Multiply the term-document matrix by the query vector
matching_titles = dense_X_example.dot(dense_query_vector.T)

# Show which titles have words in common with the query
print("\nMatching words:\n", matching_titles)

print("\nThis tells us that the first title has 3 words in common with the query,\nthe second title has one word in common, and the third title has no words in common.\n")


document_titles vector:
 [[0 1 1 0 1 0 0 0 0 0 0 1]
 [0 0 0 0 1 1 0 1 1 0 0 1]
 [1 0 0 1 0 0 1 0 0 1 1 0]]

query vector:
 [[0 1 0 0 1 0 0 0 0 0 0 1]]

Matching words:
 [[3]
 [2]
 [0]]

This tells us that the first title has 3 words in common with the query,
the second title has one word in common, and the third title has no words in common.

time: 16 ms (started: 2024-03-24 09:13:38 +01:00)


### 🚧 Todo: 
  - Look at argmax and max, 
  - Check the numpy 
      - Flatnonzero function to find the best match
      - The .A and the .flat functions
  - Show the best matching doc

#### Query

In [165]:
print("Query:", query_111, "\n")

print("Query Vector:", query_111_transformed.shape)

Query: ['how long does getting a doctorate take'] 

Query Vector: (1, 39479)
time: 0 ns (started: 2024-03-24 09:13:38 +01:00)


#### Computing similarity + finding best match - WITHOUT flatnonzero

In [166]:
# Computing the similarity
# This will give a similarity score for each document
similarity_scores = X_titles.dot(query_111_transformed.T)
print("Shape of product vector:\n", similarity_scores.shape)

# Finding the index of the highest scoring document title
most_similar_document_index = similarity_scores.argmax()
print("\nIndex of maximum similarity score:\n", most_similar_document_index)

# Now, you can get the title of the most similar document
most_similar_title = docs.iloc[most_similar_document_index]
print("\nMost similar title given the query_111:\n", most_similar_title.title, "\n")

Shape of product vector:
 (92562, 1)

Index of maximum similarity score:
 11956

Most similar title given the query_111:
 After getting my bachelors how long does it take to get a doctorate? 

time: 47 ms (started: 2024-03-24 09:13:38 +01:00)


#### Computing similarity + finding best match - WITH flatnonzero

This is the process that will be performed:

1. Get non-zero similarity scores' indices:
   
   - **indices = np.flatnonzero(dense_similarity_scores)**
   
   - This step finds all the indices of the documents that have at least one word in common with the query.

   Example: If dense_similarity_scores = [0, 2, 3, 0, 1], indices will be [1, 2, 4].

2. Flatten the similarity scores:
   
   - **flat_scores = dense_similarity_scores.flatten()**
   
   - It flattens the 2D similarity scores matrix into a 1D array to simplify finding the highest scores.

   - Example: If dense_similarity_scores was a 2D array like [[0], [2], [3], [0], [1]], flat_scores will be [0, 2, 3, 0, 1].

3. Find indices of all non-zero scores:
   
   - **non_zero_indices = np.flatnonzero(flat_scores)**

   - This is similar to step 1 but applied to the flattened array, identifying indices of all non-zero scores.

   - Example: Continuing from the previous flat_scores, non_zero_indices will also be [1, 2, 4].

4. Sort the indices by the scores:
   
   - **sorted_indices = non_zero_indices[np.argsort(flat_scores[non_zero_indices])[::-1]]**
   
   - This sorts the indices based on the scores in descending order, so the document with the highest similarity score comes first.

   - Example: np.argsort(flat_scores[non_zero_indices]) would give [2, 1, 0], which refers to scores [3, 2, 1]. After reversing with [::-1], you get [0, 1, 2], corresponding to scores [3, 2, 1] in descending order. Therefore, sorted_indices would be [2, 1, 4].

5. Retrieve the index of the best matching document:
   
   - **best_document_index = sorted_indices[0]**
   
   - This takes the first index from the sorted array, indicating the document with the most words in common with the query.

   - Example: sorted_indices is [2, 1, 4], so best_document_index is 2.

6. Retrieve the best matching document:**
   
   - **best_document = document_titles[best_document_index]**

   - Using the index found in step 5, retrieve the corresponding document title from the original list.

   - Example: If document_titles = ["title1", "title2", "title3", "title4", "title5"] and best_document_index is 2, then best_document would be "doc3".



In [167]:
import numpy as np

dense_similarity_scores = similarity_scores.A

# Get the indices of the documents with non-zero similarity scores
non_zero_indices = np.flatnonzero(dense_similarity_scores)
print("Non-zero indices within similarity score:", non_zero_indices, "\n")

# Flattening the similarity scores to a 1-D array
flat_scores = dense_similarity_scores.flatten()
print("Flattened Scores:", flat_scores, "\n")

# We can sort these indices based on the scores to find the documents with most words in common
sorted_indices = non_zero_indices[np.argsort(flat_scores[non_zero_indices])[::-1]]
print("Sorted indices:", sorted_indices)

# Not very efficient as we are sorting values

# The first element in 'sorted_indices' is the index of the document with the most words in common
best_document_index = sorted_indices[0]

# Retrieve the best matching document
best_document = docs.iloc[best_document_index]
print("\nMost similar title given the query_111:\n", best_document.title, "\n")

Non-zero indices within similarity score: [    2     7     8 ... 92549 92555 92558] 

Flattened Scores: [0 0 1 ... 0 0 0] 

Sorted indices: [11956  3685 43188 ... 55863 55861     2]

Most similar title given the query_111:
 After getting my bachelors how long does it take to get a doctorate? 

time: 46 ms (started: 2024-03-24 09:13:38 +01:00)


### 🚧 Todo: Use argpartition to get the 10 best answers

#### Information:

- np.argpartition is a function in NumPy that can be used to find the indices of the N smallest or largest values in an array efficiently. 

- It's a partial sort that rearranges the elements in the array so that the Nth element is in the place it would be if the array was fully sorted, with all smaller elements moved before it and all larger elements moved after it.

- This function is very useful when we only need the indices of the top N values and do not need the entire array to be sorted, which can save a lot of computation time.

- It sorts in ascending order by default.

- In the argument of argpartition we use -10 which tells np.argpartition that we want the indices of the ten largest values. Since np.argpartition normally returns the indices of the smallest values, by specifying -10, we're asking for the ten largest values.

- As it returns in ascending order, we select the 10 last ones to retrieve the largest ones.


In [168]:
pred10i = np.argpartition(flat_scores, -10)[-10:]
print("Top 10 indices:\n", pred10i, "\n")

Top 10 indices:
 [71758  5369 42418 83784 64965  3685 43188 91272 66310 11956] 

time: 47 ms (started: 2024-03-24 09:13:38 +01:00)


In [169]:
docs.iloc[pred10i]

,docid,title,body
71761,D1531333,Top 30 Doctor insights on: How Long Does It Ta...,Top 30 Doctor insights on: How Long Does It Ta...
5370,D1279758,Top 30 Doctor insights on: How Long Does A Her...,Top 30 Doctor insights on: How Long Does A Her...
42420,D832615,How long does it take to cook pork chops?,Pork How Long Does X Take? Food Safety Meat Co...
83787,D1951655,How long does it take to get a Doctorate of Nu...,Answers.com ® Wiki Answers ® Categories Jobs &...
64968,D3083629,How long does it take to become comfortable wi...,Answers.com ® Wiki Answers ® Categories Techno...
3686,D2111066,After getting a flu shot how long does it take...,Celticballa568 19 Contributions After getting ...
43190,D700799,Holidays By Country Â»Holidays By Religion Â»O...,Home > Important Days > Daylight Savings > Day...
91275,D3090762,How Long Does It Take to Get a Doctorate in En...,Home » College How Long Does It Take to Get a ...
66313,D2956542,How long does it take to finish a doctorate?,Education & Reference Higher Education (Univer...
11957,D2408424,After getting my bachelors how long does it ta...,Education & Reference Higher Education (Univer...


time: 15 ms (started: 2024-03-24 09:13:38 +01:00)


In [170]:
docs.iloc[pred10i].docid

71761    D1531333
5370     D1279758
42420     D832615
83787    D1951655
64968    D3083629
3686     D2111066
43190     D700799
91275    D3090762
66313    D2956542
11957    D2408424
Name: docid, dtype: object

time: 16 ms (started: 2024-03-24 09:13:38 +01:00)


In [171]:
gold[gold.qid==251898]#.docid

,qid,docid,rank,score
36200,251898,D2865964,1,-4.74293
36201,251898,D3557816,2,-4.90695
36202,251898,D2723985,3,-4.95911
36203,251898,D1951655,4,-4.97272
36204,251898,D1709749,5,-5.02176
...,...,...,...,...
36295,251898,D2531901,96,-5.56896
36296,251898,D2956542,97,-5.57138
36297,251898,D301873,98,-5.57262
36298,251898,D2952336,99,-5.57504


time: 16 ms (started: 2024-03-24 09:13:38 +01:00)


### 🚧 Todo:
- Find the relevant documents that are in our top 10
- User intersect1d
- Compute the precision p@10

#### Relevant Documents

In [172]:
# Top 10 predictions we found using CountVectorizer
top_10_predictions = docs.iloc[pred10i].docid

# Actual gold target
gold_docids = gold[gold.qid==251898].docid

# We use intersect1d to find common docids
relevant_docids = np.intersect1d(top_10_predictions, gold_docids)
relevant_docids

array(['D1951655', 'D2408424', 'D2956542', 'D3083629', 'D3090762'],
      dtype=object)

time: 16 ms (started: 2024-03-24 09:13:38 +01:00)


#### P@10

- Precision at K (P@K) measures the proportion of relevant documents in the top-K results of a query. When K is set to 10, this is known as P@10.

In [173]:
# The number of relevant documents in the top 10
num_relevant_docids = len(relevant_docids)

# Precision at 10 is the number of relevant documents in the top 10 divided by 10
p_at_10 = num_relevant_docids / 10.0

print("P@10:", p_at_10)

P@10: 0.5
time: 0 ns (started: 2024-03-24 09:13:38 +01:00)


#### Precision Function

In [174]:
queries[queries.qid == 251898]["query"].iloc[0]

'how long does getting a doctorate take'

time: 0 ns (started: 2024-03-24 09:13:38 +01:00)


In [178]:
# 🚧 Todo: Build a function p@10 that gives the precision at 10

def pAt10(X, gold, qid):

    # Step 1: Get the query text for the given query_id
    query_text = queries[queries.qid == qid]["query"].iloc[0]
    
    # Step 2: Transform the query into the vector space
    query_vector = vectorizer.transform([query_text])

    # Step 3: Compute similarity scores for all documents
    similarity_scores = X.dot(query_vector.T).toarray().flatten()

    # Step 4: Get the indices of the top 10 documents
    top10_indices = np.argpartition(similarity_scores, -10)[-10:]
    top10_indices_sorted = top10_indices[np.argsort(similarity_scores[top10_indices])[::-1]]
    top10_docids = docs.iloc[top10_indices_sorted]['docid'].values

    # Step 5: Get the relevant docids from the gold standard for the given query_id
    relevant_docids = gold[gold['qid'] == qid]['docid'].values

    # Step 6: Calculate P@10
    relevant_in_top_10 = np.intersect1d(top10_docids, relevant_docids)
    p_at_10 = len(relevant_in_top_10) / 10.0

    return p_at_10

# Usage:
X_titles = vectorizer.fit_transform(docs.title.values)
pAt10_score = pAt10(X_titles, gold, 251898)
print(f"P@10 for query ID 251898: {pAt10_score}")



P@10 for query ID 251898: 0.5
time: 1.84 s (started: 2024-03-24 09:15:56 +01:00)


### 🚧 Todo:
- Take our 500 training queries qid
- Apply our function
- Compute the average

#### Training Queries

In [179]:
training_queries.qid

0      687888
1      480210
2      591004
3      260536
4       39422
        ...  
495    133970
496     79788
497    791583
498    732078
499    197098
Name: qid, Length: 500, dtype: int64

time: 16 ms (started: 2024-03-24 09:16:32 +01:00)


#### List of p@10 results for training queries

In [180]:
training_queries_pAt10 = [pAt10(X_titles, gold, qid) for qid in training_queries.qid]
#training_queries_pAt10

time: 8.48 s (started: 2024-03-24 09:16:34 +01:00)


#### Average Result

In [181]:
avg_pAt10 = np.mean(training_queries_pAt10)
avg_pAt10

0.319

time: 16 ms (started: 2024-03-24 09:16:47 +01:00)


#### Best Prediction

- that looks like a baseline we can beat :)
- what's the query we are doing best in?
    - max?

In [182]:
best_pred = np.argmax(training_queries_pAt10)
best_query_prediction = training_queries.iloc[best_pred]
best_query_prediction.query

'define standard operating procedure template'

time: 16 ms (started: 2024-03-24 09:16:49 +01:00)


#### Checking that the result makes sense

In [183]:
# Repeating the same process as before

best_query_vector = vectorizer.transform([best_query_prediction.query])
similarity_scores = X_titles.dot(best_query_vector.T)
dense_similarity_scores = similarity_scores.A
flat_scores = dense_similarity_scores.flatten()
pred10i = np.argpartition(flat_scores, -10)[-10:]
docs.iloc[pred10i]

,docid,title,body
39209,D211341,Standard operating procedure,"For the 2008 documentary film, see Standard Op..."
12948,D836561,SOP - Standard Operating Procedure,SOP - Standard Operating Procedure SOP - How t...
38937,D3557178,How to Create an SOP Standard Operating Proced...,How to Create an SOP Standard Operating Proced...
33465,D3557177,How Toâ¦ Writing Standard Operating Procedures...,by Chris Anderson / in Save Time Writing Proce...
39949,D3458106,standard operating procedure,standard operating procedure Also found in: Th...
36121,D1068918,Sample Standard Operating Procedure...,Sample Standard Operating Procedure... The fol...
61004,D1068919,37 Best Standard Operating Procedure (SOP) Tem...,For organizations or institutions which desire...
77230,D1068921,Standard Operating Procedure Template â 13+ Fr...,Every company has got a host of things to do a...
25297,D2555909,Standard Operating Procedure Template,What is it: A Standard Operating Procedure (SO...
78890,D2931233,SOP Template - Standard Operating Procedure Te...,SOP Template - Standard Operating Procedure Te...


time: 32 ms (started: 2024-03-24 09:16:51 +01:00)


- oh, we have just been lucky before...

## 🚧 Todo:

- Redo the vectorization and evaluation on the whole text, not only the titles - DONE
- Try the non-binary CountVectorizer - DONE
- Go for tf-idf - DONE
    - Play with at least two options and re-evaluate - DONE
- Find other improvements. these may include:
    - Cleaning the text - DONE
    - Heuristically combining title and body matches - DONE
    - Looking at bigrams - Tried, but PC failed miserably
    - Looking at terms (by means of a clean multi-word term list from wikipedia, see notebook 1) - Not Tried
    - By removing stopwords (look at nltk or spacy to do that) - DONE
    - Trying an implementation of bm25 (slow with standard implementation, see last cell if you want to do it) - DONE
  
- $\textcolor{red}{\text{Do a grid search with a few promising parameters}}$
    - Maybe get inspired by GridSearchCV and pipelines in:
        - https://scikit-learn.org/stable/auto_examples/model_selection/grid_search_text_feature_extraction.html#sphx-glr-auto-examples-model-selection-grid-search-text-feature-extraction-py
    - You can also check the weel-written section "Pipelines" in this book:
        - https://www.oreilly.com/library/view/applied-text-analysis/9781491963036/ch04.html
    - $\textcolor{red}{\text{Make a nice visualization of the results}}$
    
- Interpret the complete results in 3 to 5 sentences. - DONE
    - $\textcolor{red}{\text{What strategy would do best if we switch our evaluation to p@100?}}$

- Give some ideas for improving the results - DONE





## CountVectorizer

### Binary Vectorization on the whole text (title + body) + Evaluation on p@10

##### Reminder

In [184]:
docs

,docid,title,body
0,D2981241,What do you call a group of lions?,Lions Vocabulary of the English Language Word ...
1,D687756,.,"The A Priori Argument ( also, Rationalization;..."
2,D913099,Everything You Need To Learn How To Cook Veget...,Home > How To Cook Vegetables Everything You N...
3,D328017,"What is the difference between latitude, longi...",Longitude Latitude Geographic Coordinate Syste...
4,D1636347,When was the pulley invented?,Answers.com ® Wiki Answers ® Categories Techno...
...,...,...,...
92560,D3379210,Top 39 Doctor insights on: Can An Iud Cause Ha...,Top 39 Doctor insights on: Can An Iud Cause Ha...
92561,D3068739,How to get back your DirecTV cancellation fees,How to get back your Direc TV cancellation fee...
92562,D1590402,Certification FAQs,Fingerprinting 1. Where can I get fingerprinte...
92563,D2175490,Greenhouse gas emissions by Canadian economic ...,"Access PDF (682 KB)In 2015, Canada's total gre..."


time: 16 ms (started: 2024-03-24 09:16:55 +01:00)


#### Creating Whole_text

In [12]:
docs["whole_text"] = docs["title"] + " " + docs["body"]
docs

C:\Users\pmoll\AppData\Local\Temp\ipykernel_14472\3491648862.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  docs["whole_text"] = docs["title"] + " " + docs["body"]


,docid,title,body,whole_text
0,D2981241,What do you call a group of lions?,Lions Vocabulary of the English Language Word ...,What do you call a group of lions? Lions Vocab...
1,D687756,.,"The A Priori Argument ( also, Rationalization;...",". The A Priori Argument ( also, Rationalizatio..."
2,D913099,Everything You Need To Learn How To Cook Veget...,Home > How To Cook Vegetables Everything You N...,Everything You Need To Learn How To Cook Veget...
3,D328017,"What is the difference between latitude, longi...",Longitude Latitude Geographic Coordinate Syste...,"What is the difference between latitude, longi..."
4,D1636347,When was the pulley invented?,Answers.com ® Wiki Answers ® Categories Techno...,When was the pulley invented? Answers.com ® Wi...
...,...,...,...,...
92560,D3379210,Top 39 Doctor insights on: Can An Iud Cause Ha...,Top 39 Doctor insights on: Can An Iud Cause Ha...,Top 39 Doctor insights on: Can An Iud Cause Ha...
92561,D3068739,How to get back your DirecTV cancellation fees,How to get back your Direc TV cancellation fee...,How to get back your DirecTV cancellation fees...
92562,D1590402,Certification FAQs,Fingerprinting 1. Where can I get fingerprinte...,Certification FAQs Fingerprinting 1. Where can...
92563,D2175490,Greenhouse gas emissions by Canadian economic ...,"Access PDF (682 KB)In 2015, Canada's total gre...",Greenhouse gas emissions by Canadian economic ...


time: 25.5 s (started: 2024-03-24 19:31:26 +01:00)


#### Whole text

In [189]:
# 1. Vectorize the whole text
vectorizer = CountVectorizer(binary=True)
X_whole_text = vectorizer.fit_transform(docs['whole_text'])
#5.43s

time: 5min 43s (started: 2024-03-24 09:33:46 +01:00)


##### Example - Single Query

In [190]:
pAt10_score = pAt10(X_whole_text, gold, 251898)
print(f"P@10 for query ID 251898: {pAt10_score}")

P@10 for query ID 251898: 0.1
time: 1.75 s (started: 2024-03-24 09:39:29 +01:00)


##### Training Queries

In [191]:
# Evaluation part: p@10
training_queries_pAt10_whole = [pAt10(X_whole_text, gold, qid) for qid in training_queries.qid]
avg_pAt10_whole = np.mean(training_queries_pAt10_whole)
avg_pAt10_whole

0.4816

time: 7min 27s (started: 2024-03-24 09:39:31 +01:00)


#### Conclusion: CountVectorizer(binary=True) only Titles vs Whole Text

- #### Remark 1:

    We appreciate a decrease in P@10 when moving from title-only (0.5) to whole-text (0.1) vectorization when it comes to predicting **a single query (251898)** which can happen for several reasons:

    1. $\underline{\text{Noisy Data:}}$ Titles are generally concise and highly relevant to the content of the document, but the body might include a lot of additional information, some of which could be considered noise in the context of the specific query. This noise might dilute the relevance signals that were clear in the title alone.

    2. $\underline{\text{Keyword Dilution:}}$ When using the entire text, the frequency of key terms from the title might be diluted among many other terms, leading to less distinctive vectors for relevance ranking.

    3. $\underline{\text{Overfitting to Titles:}}$ The model or the baseline might be overfitting to the style or keywords found in titles. When you add the body text, the model's ability to distinguish relevant from non-relevant documents might decrease because it's now relying on features that are not as strongly indicative of relevance.

    4. $\underline{\text{Vector Space Dimensionality:}}$ Including the full text increases the dimensionality of the vector space significantly (since you likely have more unique words in the full texts than in the titles), which can sometimes lead to poorer performance if not managed correctly, a phenomenon sometimes referred to as the "curse of dimensionality."

- #### Remark 2:

    However and unexpectedly, the average P@10 score improves when using the whole text (0.4816) instead of just the titles (0.319) for a training set of 500 queries, this suggests that including the body of the documents provides a more comprehensive set of features that help in identifying relevant documents. Here are some possible reasons for this improvement:

    1. **Richer Context**: Whole text contains more context and information than titles alone, which can lead to better matching with queries, especially for complex queries that require more context to interpret correctly.

    2. **Detailed Content**: The body of a document often contains detailed discussions of topics that might only be briefly mentioned in the title. This can help in capturing the relevance for queries that are more specific and detailed.

    3. **Keyword Occurrence**: Important keywords may occur in the body text that do not appear in the titles. These occurrences can contribute to a more accurate representation of the document's content and hence improve the matching with queries.

    4. **Diverse Vocabulary**: The body text introduces a wider and more diverse vocabulary. This can help differentiate documents more precisely and match a broader range of queries.

    5. **Long-tail Queries**: For more descriptive or long-tail queries, the chance that the relevant terms appear in the body rather than just the title is higher. Thus, the whole text provides a better chance of matching these queries.

    6. **Overfitting to Training Set**: It's also possible that the model has become better at matching the patterns in the training set specifically when using the whole text, which might not generalize as well to unseen queries or a different test set.

    In summary, while the P@10 score might decrease for individual queries when using the whole text, the average P@10 score across a large set of queries can improve if the additional context and information contained in the full text leads to better overall matching between queries and documents.

### Non-Binary (Frequency) Vectorization on the whole text (title + body) + Evaluation on p@10

#### Title only

In [203]:
# Vectorize the whole text
vectorizer = CountVectorizer(binary=False)
X_titles_non_binary = vectorizer.fit_transform(docs['title'])

time: 1.33 s (started: 2024-03-24 10:33:15 +01:00)


##### Example - Single Query

In [204]:
pAt10_score = pAt10(X_titles_non_binary, gold, 251898)
print(f"P@10 for query ID 251898: {pAt10_score}")

P@10 for query ID 251898: 0.0
time: 16 ms (started: 2024-03-24 10:33:17 +01:00)


##### Training Queries

In [205]:
# Evaluation part: p@10
training_queries_pAt10_titles_non_binary = [pAt10(X_titles_non_binary, gold, qid) for qid in training_queries.qid]
avg_pAt10_titles_non_binary = np.mean(training_queries_pAt10_titles_non_binary)
avg_pAt10_titles_non_binary

0.046400000000000004

time: 4.2 s (started: 2024-03-24 10:33:20 +01:00)


#### Whole Text

In [195]:
# Vectorize the whole text
vectorizer = CountVectorizer(binary=False)
X_non_binary = vectorizer.fit_transform(docs['whole_text'])

time: 6min 12s (started: 2024-03-24 10:07:19 +01:00)


##### Example - Single Query

In [196]:
pAt10_score_non_binary = pAt10(X_non_binary, gold, 251898)
print(f"P@10 for query ID 251898: {pAt10_score_non_binary}")

P@10 for query ID 251898: 0.0
time: 1.58 s (started: 2024-03-24 10:13:32 +01:00)


##### Training Queries

In [197]:
# Evaluation part: p@10
training_queries_pAt10_whole_non_binary = [pAt10(X_non_binary, gold, qid) for qid in training_queries.qid]
avg_pAt10_whole_non_binary = np.mean(training_queries_pAt10_whole_non_binary)
avg_pAt10_whole_non_binary

0.046

time: 7min 28s (started: 2024-03-24 10:13:34 +01:00)


#### Conclusion: CountVectorizer(binary=False) only Titles vs Whole Text

- #### Remark 1:

   These outcomes can seem counterintuitive, but they highlight how the choice of text representation can significantly impact the performance of information retrieval systems. Let's consider the possible reasons behind these results:

1. **CountVectorizer (binary=False) with Whole Text (P@10=0.046 for the training set, 0.0 for a single query):**
   
   - When binary=False, the CountVectorizer accounts for term frequency. This can be detrimental if the frequency isn't normalized because common words may dominate the feature representation.
   
   - Including the whole text likely introduces a lot of common words and phrases, which might not be relevant to the query's intent, leading to a diluted signal for relevance.

2. **CountVectorizer (binary=False) with Titles (P@10=0.046 for the training set, 0.0 for a single query):**
   
   - The same frequency issue applies, but because titles are shorter and more focused, we might expect better performance. However, if the gold standard heavily favors certain keywords or phrases that are lost in the non-binary representation, performance can suffer as seen here.

3. **CountVectorizer (binary=True) with Whole Text (P@10=0.4816 for the training set, 0.1 for a single query):**
   
   - Surprisingly, using a binary representation with the whole text has performed quite well for the training set. This might indicate that the mere presence of certain words, regardless of their frequency, is a strong indicator of relevance across your queries.
   
   - For the single query, the score is much lower (0.1), suggesting this query might be an outlier or not well represented in the training set.

4. **CountVectorizer (binary=True) with Titles (P@10=0.319 for the training set, 0.5 for a single query):**
   
   - This is the best-performing scenario for the single query, indicating that specific keywords in the titles are highly relevant for this query.
   
   - For the training set, while still decent, it’s not as high as with the whole text, possibly due to the reduced context that titles provide, which might not capture the full scope of relevance.


Key Insights:

- **Model Sensitivity to Binary vs. Frequency Counts:** This indicates that for some queries, particularly the one I chose as a test (Query 251898), the binary presence or absence of terms (binary=true) is more indicative of relevance than the frequency of terms (binary=false).

- **Importance of Title vs. Whole Text:** Titles can often be quite descriptive of the main content, but they might lack some context that the body of the text provides. However, including the full text may introduce noise, which can be particularly problematic if not managed by proper normalization or term weighting like TF-IDF, which we will consider later on the Notebook.

- **Evaluation Set Disparity:** The disparity between the performance on a single query versus the training set of 500 queries suggests there might be variability in how well each query is represented in the dataset or that some queries are outliers.


Improvement Steps:

- **Normalize Frequency Counts:** If using term frequency (binary=false), consider normalizing term counts or using TF-IDF to mitigate the impact of frequently occurring words.


## TF-IDF

### Model Evaluation

In [16]:
# This function will vectorize the documents, transform the queries, and calculate P@10
def evaluate_model(text, vectorizer, docs, queries, gold_standard):
    # Fit the vectorizer on the documents' whole text
    X_docs = vectorizer.fit_transform(docs[text])

    # Initialize a list to store P@10 for each query
    p_at_10_scores = []

    # Loop through all queries
    for idx, row in tqdm(queries.iterrows()):
        query_id = row['qid']
        query_text = row['query']

        # Transform the query to fit the vector space of the document terms
        query_vector = vectorizer.transform([query_text])

        # Calculate similarity scores
        similarity_scores = X_docs.dot(query_vector.T).toarray().flatten()

        # Get the indices of the top 10 documents
        top10_indices = np.argpartition(similarity_scores, -10)[-10:]
        top10_indices_sorted = top10_indices[np.argsort(similarity_scores[top10_indices])[::-1]]
        top10_docids = docs.iloc[top10_indices_sorted]['docid'].values

        # Get the relevant docids from the gold standard for the query_id
        relevant_docids = gold_standard[gold_standard['qid'] == query_id]['docid'].values

        # Calculate P@10 for the query
        relevant_in_top_10 = np.intersect1d(top10_docids, relevant_docids)
        p_at_10 = len(relevant_in_top_10) / 10.0
        p_at_10_scores.append(p_at_10)

    # Calculate the average P@10
    average_p_at_10 = sum(p_at_10_scores) / len(p_at_10_scores)
    return average_p_at_10

time: 0 ns (started: 2024-03-24 20:10:37 +01:00)


### TF-IDF with whole_text - Option 1

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Instantiation of TF-IDF - Option 1
tfidf_vectorizer_option1 = TfidfVectorizer(max_features=50000)

# Option 1 evaluation
average_p_at_10_option1 = evaluate_model("whole_text", tfidf_vectorizer_option1, docs, training_queries, gold)
print(f'Average P@10 for option 1: {average_p_at_10_option1}')


0it [00:00, ?it/s]

Average P@10 for option 1: 0.9003999999999991
time: 11min 25s (started: 2024-03-24 14:04:41 +01:00)


### TF-IDF ignoring terms - Option 2

Option 2: Using unigrams and bigrams, ignoring terms that appear in more than 70% of the documents, and terms that appear in only 2 document.

- I tried but my computer doesn't have enough memory

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Instantiation of TF-IDF - Option 2
tfidf_vectorizer_option2 = TfidfVectorizer(
                            ngram_range= (1,2),  # Include bigrams
                            max_df=0.7,          # Ignore terms that appear in more than 70% of the documents
                            min_df=2,            # Ignore terms that appear in fewer than 2 documents
                            max_features=30000   # Limit the vocabulary size to the top 30,000 terms
                        )
# Option 2 evaluation
# average_p_at_10_option2 = evaluate_model("whole_text", tfidf_vectorizer_option2, docs, training_queries, gold)
# print(f'Average P@10 for option 2: {average_p_at_10_option2}')

time: 344 ms (started: 2024-03-24 15:26:18 +01:00)


In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Instantiation of TF-IDF - Option 2
tfidf_vectorizer_option2 = TfidfVectorizer(
                            max_df=0.7,          # Ignore terms that appear in more than 70% of the documents
                            min_df=2,            # Ignore terms that appear in fewer than 2 documents
                            max_features=55000   # Limit the vocabulary size to the top 30,000 terms
                        )
# Option 2 evaluation
average_p_at_10_option2 = evaluate_model("whole_text", tfidf_vectorizer_option2, docs, training_queries, gold)
print(f'Average P@10 for option 2: {average_p_at_10_option2}')

0it [00:00, ?it/s]

Average P@10 for option 2: 0.8995999999999992
time: 5min 27s (started: 2024-03-24 15:26:41 +01:00)


## Other Improvements

### Cleaning Text

The following cleaning techniques will be applied to the whole_text (title + body).
    
- Replace new lines + line breaks with space
- Lowering the text
- Remove punctuation and numbers
- Tokenization by NLTK
- Removing English stopwords
- Applying stemming

Here’s an example of how stemming works with the NLTK's PorterStemmer:

- The word "running" would be stemmed to "run".
- "Happily" would be stemmed to "happi".
- "Better" might be stemmed to "better" or "good" depending on the stemmer's algorithm and if it's aware of such irregular forms.

In [13]:
def clean_text(text):
    # Replace new lines and line breaks with space
    text = text.replace('\n', ' ').replace('\r', ' ')
    
    # Convert to lowercase
    text = text.lower()
    
    # Remove punctuation and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    # Tokenize
    tokens = word_tokenize(text)
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    
    # Stemming (optional)
    #stemmer = PorterStemmer()
    #tokens = [stemmer.stem(token) for token in tokens]
    return ' '.join(tokens)


time: 31 ms (started: 2024-03-24 19:31:52 +01:00)


### Creation of Cleaned_Text

In [14]:
tqdm.pandas()

# Applying the function to whole_text within docs
docs['cleaned_text'] = docs['whole_text'].progress_apply(clean_text)


  0%|          | 0/92562 [00:00<?, ?it/s]

time: 27min 56s (started: 2024-03-24 19:31:59 +01:00)


C:\Users\pmoll\AppData\Local\Temp\ipykernel_14472\3855620958.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  docs['cleaned_text'] = docs['whole_text'].progress_apply(clean_text)


### TF-IDF with cleaned_text - Option 3

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Instantiation of TF-IDF - Option 3
tfidf_vectorizer_option3 = TfidfVectorizer(max_features=50000)

# Option 3 evaluation
average_p_at_10_option1 = evaluate_model("cleaned_text", tfidf_vectorizer_option3, docs, training_queries, gold)
print(f'Average P@10 for option 3: {average_p_at_10_option1}')


0it [00:00, ?it/s]

Average P@10 for option 3: 0.8847999999999996
time: 4min 58s (started: 2024-03-24 20:10:41 +01:00)


### TF-IDF with Heuristically combining (cleaned(Title) x2 + cleaned(Body)) - Option 4

In [20]:
# Defining a function for heuristic combination
def combine_title_body(title, body, title_weight=2):
    # Clean the title and body
    cleaned_title = clean_text(title)
    cleaned_body = clean_text(body)
    # Duplicate title terms based on the weight
    combined_text = (cleaned_title + ' ') * title_weight + cleaned_body
    return combined_text

# Apply the combination function to each row
docs['heuristic_combined'] = docs.progress_apply(lambda row: combine_title_body(row['title'], row['whole_text']), axis=1)


  0%|          | 0/92562 [00:00<?, ?it/s]

time: 31min 49s (started: 2024-03-24 20:20:05 +01:00)


C:\Users\pmoll\AppData\Local\Temp\ipykernel_14472\4202488366.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  docs['heuristic_combined'] = docs.progress_apply(lambda row: combine_title_body(row['title'], row['whole_text']), axis=1)


In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Instantiation of TF-IDF - Option 4
tfidf_vectorizer_option4 = TfidfVectorizer(max_features=50000)

# Option 4 evaluation
average_p_at_10_option1 = evaluate_model("heuristic_combined", tfidf_vectorizer_option4, docs, training_queries, gold)
print(f'Average P@10 for option 4: {average_p_at_10_option1}')

0it [00:00, ?it/s]

Average P@10 for option 4: 0.8883999999999991
time: 6min 7s (started: 2024-03-24 17:43:36 +01:00)


## BM25 - Option 5

### Class BM25

In [21]:
from sklearn.base import BaseEstimator, ClassifierMixin
import numpy as np
import gc

class BM25Score(BaseEstimator, ClassifierMixin):
    def __init__(self, k1=1.5, b=0.75): # vectorized_docs: word counts
        self.k1 = k1
        self.b = b
        # self.vectorized_docs will be set in the fit method


    def fit(self, X, y=None):
        self.vectorized_docs = X
        self.n_d = self.vectorized_docs.sum(axis=1).A.flatten()
        self.avgdl = np.mean(self.n_d)
        self.n_docs = self.vectorized_docs.shape[0]

        # Calculate IDF
        document_frequency = (self.vectorized_docs > 0).sum(axis=0).A.flatten()
        self.idf = np.log((self.n_docs - document_frequency + 0.5) / (document_frequency + 0.5) + 1)
        return self

    def predict(self, X):
        # bigger batches => faster but more memory heavy
        n_queries = X.shape[0]
        batch_size = 50
        final_scores = []

        for batch_idx in range(0, n_queries, batch_size):
            end_idx = min(batch_idx + batch_size, n_queries)
            batch_vectorized_queries = X[batch_idx:end_idx]
            idx_tokens = np.flatnonzero(batch_vectorized_queries.sum(axis=0))
            batch_vectorized_queries = batch_vectorized_queries[:, idx_tokens]
            vectorized_docs = self.vectorized_docs[:, idx_tokens].toarray()
            idf = self.idf[idx_tokens]

            scores = idf * (
                (vectorized_docs * (self.k1 + 1)) /
                (vectorized_docs + self.k1 * (1 - self.b + self.b * (self.n_d[:, None] / self.avgdl)))
            )
            final_scores.append(((batch_vectorized_queries > 0).astype(np.float32) @ scores.T))

            del vectorized_docs, scores
            gc.collect()

        return np.concatenate(final_scores, axis=0)


time: 282 ms (started: 2024-03-24 20:51:56 +01:00)


### Vectorization + Cleaning

In [25]:
from sklearn.feature_extraction.text import CountVectorizer
 
# Vectorize the whole text
vectorizer = CountVectorizer(binary=True)
X_final = vectorizer.fit_transform(docs['cleaned_text'])

# To use the BM25 score for ranking, we vectorize all queries and use the predict method
cleaned_queries = []
for query in tqdm(training_queries['query']):
    cleaned_queries.append(clean_text(query))


  0%|          | 0/500 [00:00<?, ?it/s]

time: 3min 3s (started: 2024-03-24 21:21:26 +01:00)


### Prediction

In [26]:
# Instantiation of BM25Score + fitting + prediction
bm25 = BM25Score()
bm25.fit(X_final)
queries_vectors = vectorizer.transform(cleaned_queries)
scores = bm25.predict(queries_vectors)

time: 24 s (started: 2024-03-24 21:25:14 +01:00)


In [27]:
scores.shape

(500, 92562)

time: 16 ms (started: 2024-03-24 21:25:39 +01:00)


In [28]:
def evaluate_model_bm25_optimized(scores, docs, queries, gold_standard):
    # Assuming 'scores' is a 2D array: rows are queries, columns are documents.
    p_at_10_scores = []
    
    for idx, row in enumerate(queries.itertuples()):
        query_id = getattr(row, 'qid')
        
        # Get scores for the current query
        query_scores = scores[idx]
        
        # Efficiently find the indices of the top 10 scores for this query
        top10_indices = np.argpartition(query_scores, -10)[-10:]
        top10_indices_sorted = top10_indices[np.argsort(query_scores[top10_indices])[::-1]]
        
        # Assuming doc_ids is a list of document IDs that aligns with columns in 'scores'
        top10_docids = [docs.iloc[idx]['docid'] for idx in top10_indices_sorted]
        
        # Get the relevant docids for the query_id from gold_standard
        relevant_docids = gold_standard[gold_standard['qid'] == query_id]['docid'].values
        
        # Calculate P@10 for the query
        relevant_in_top_10 = np.intersect1d(top10_docids, relevant_docids)
        p_at_10 = len(relevant_in_top_10) / 10.0
        p_at_10_scores.append(p_at_10)
    
    # Calculate the average P@10
    average_p_at_10 = np.mean(p_at_10_scores)
    return average_p_at_10



average_p_at_10 = evaluate_model_bm25_optimized(scores, docs, training_queries, gold)
print(f"Average BM25 P@10: {average_p_at_10}")


Average BM25 P@10: 0.8438
time: 3.12 s (started: 2024-03-24 21:25:40 +01:00)


## Notebook's Detailed Conclusion

Based on the P@10 results for different text processing and vectorization approaches, we can draw several conclusions and identify potential areas for improvement. The results indicate varying levels of effectiveness in capturing the relevance of documents to queries, which is crucial for information retrieval tasks. Here's a summary and analysis, following the order of performance:

1. **TF-IDF with whole text (not cleaned) + max_df = 0.7, min_df = 2 and max_features = 55000 obtained 0.899**: This approach yielded the best performance, suggesting that the inclusion of both frequent and infrequent terms (controlled by max_df and min_df) in a relatively large feature set (max_features=55000) captures a rich set of relevance signals, even without explicit text cleaning.

2. **TF-IDF with heuristic_combined (cleaned(title)x2 + cleaned(body)) obtained 0.888 with max_features = 50000**: Close to the top performer, this strategy emphasizes title terms by doubling their presence, which intuitively boosts the weight of potentially more relevant terms found in titles. Cleaning the text also seems to contribute positively, removing noise that might dilute important signals.

3. **TF-IDF with cleaned_text obtained 0.885 with max_features = 50000**: Slightly underperforming compared to the heuristic combination, directly cleaning the text without specific emphasis on titles still yields high performance, reinforcing the value of preprocessing steps in enhancing the quality of the feature space.

4. **BM25 with cleaned_text obtained 0.843**: As a well-regarded algorithm for information retrieval, BM25's lower performance compared to the top TF-IDF configurations might be due to the particular characteristics of the dataset or the choice of k1 and b parameters. Nonetheless, its strong performance underscores the effectiveness of relevance-based scoring mechanisms.

5. **Binary vectorization on whole_text obtained 0.482**: Representing a significant drop in effectiveness, binary vectorization on the whole text suggests that mere presence/absence signals without frequency or TF-IDF weighting are less informative for relevance determination.

6. **Binary vectorization on titles obtained 0.319**: Binary vectorization's lower performance on titles alone highlights the challenge of relying solely on the presence or absence of terms in titles, which may not capture enough contextual or semantic information for accurate relevance scoring.

7. **Non-binary vectorization on whole_text and titles both obtained 0.046**: Surprisingly, treating term frequencies in a non-binary manner for both whole texts and titles underperformed significantly. This suggests that without proper normalization or weighting, raw frequencies may introduce noise or overemphasize less relevant terms, obscuring meaningful relevance signals.

### Possible Improvements:

- **Feature Selection and Dimensionality**: I would experiment with different values for `max_features`, `max_df`, and `min_df` to fine-tune the balance between including informative terms and excluding noise. Advanced dimensionality reduction techniques could also be explored.
  
- **Parameter Tuning for BM25**: It could be adjusted BM25's `k1` and `b` parameters could potentially improve its performance to better reflect the dataset's characteristics.

- **Combining Signals**: Leveraging a combination of different vectorization and scoring methods could harness the strengths of each approach. Ensemble methods or machine learning models that can learn from multiple feature sets might offer enhanced performance.

- **Semantic Understanding**: Exploring embeddings that capture deeper semantic meanings, such as those from language models (e.g., BERT), could provide more nuanced relevance signals.

The results are intriguing and highlight the importance of methodical experimentation with text processing and vectorization techniques in information retrieval tasks. The success of TF-IDF approaches, especially with careful feature selection and text preprocessing, is noteworthy, though the strong showing of BM25 also underscores the value of relevance-focused scoring methods.

## Notebook's Concise Conclusion

- The evaluation of different text vectorization and scoring methods revealed that TF-IDF with uncleaned whole text and a large feature set performed the best (P@10=0.899), highlighting the importance of comprehensive feature representation and careful parameter tuning. 

- Close behind were methods emphasizing title relevance through heuristic combinations and cleaned text, suggesting that strategic emphasis on specific document parts and preprocessing can enhance relevance detection. 

- Surprisingly, BM25, a standard in information retrieval, scored lower, indicating potential for parameter optimization. 

- Binary vectorization methods, especially when applied to titles alone or the whole text, showed significantly lower effectiveness, underscoring the value of frequency and TF-IDF weighting over mere presence/absence signals. 

- These results underscore the nuanced impact of text preprocessing, feature selection, and scoring mechanisms on information retrieval performance, advocating for a balanced approach that combines strategic emphasis on document components with robust scoring algorithms.